Data Preparation for KGE datasets

In [ ]:
pip install wikipedia2vec

In [ ]:
from wikipedia2vec import Wikipedia2Vec
wiki2vec = Wikipedia2Vec.load("enwiki_20180420_100d.pkl")

In [ ]:
import pandas as pd


if dataset == 1:
  # Source: https://www.kaggle.com/jessicali9530/kuc-hackathon-winter-2018
  df = pd.read_csv("drugsComTest_raw.csv")
  conditions = df_new['condition'].value_counts()[:10].index.to_list()
  df_int = pd.DataFrame(columns = df_new.columns.tolist())
  for condition in conditions:
    df_int =  pd.concat([df_int,d[df['condition']==condition]])
  drugs = []
  drug_list = df_int['drugName'].values
  drug_list = list(drug_list)
  for drug in drug_list:
    drugs.append(drug)
  drug_list = list(drugs)
  drug_set = set(drug_list)
  count = 0
  drug_dict={}
  drug_list_i = []
  for drug in drug_set:
    try:
      drug_dict[drug] = np.array(wiki2vec.get_entity_vector(drug))
    except KeyError:
      count = count + 1
      drug_list_i.append(drug) 

  drug_set_new = set(drug_list_i)
  df_remove = pd.DataFrame(columns = df_int.columns.tolist())
  for drug in drug_set_new:
    df_remove =  pd.concat([df_remove,df_int[df_int['drugName']==drug]]) 
  df_int = df_int[~df_int.isin(df_remove)].dropna()
  df_int = df_int.loc[:,['drugName','condition','review']]
  df_final = pd.DataFrame(columns = df_int.columns.tolist())
  for condition in conditions:
    df_final =  pd.concat([df_final,df_int[df_int['condition']==condition][:500]]) 
  category_dict = {"Insomnia":0,"ADHD":1,"Obesity":2,"Weight Loss":3,"Acne":4,"Depression":5,"Bipolar Disorde":6,"Anxiety":7,"Birth Control":8,"Pain":9}
  df_final["Label"] = df_final["condition"].map(category_dict)
  df_final = df_final.sample(frac=1)
  df_final.columns = ['drugName','condition','review','Labels']
  df_final.to_csv('med_500.csv', index = False)
elif dataset == 2: 
  # Source: https://www.kaggle.com/jrobischon/wikipedia-movie-plots
  df = pd.read_csv("wiki_movie_plots_deduped.csv")
  df_new = df.loc[df['Director'] != "Unknown"]
  df_new = df_new.loc[df_new['Genre'] != "unknown"]
  genres = df_new['Genre'].value_counts()[:5].index.to_list()
  df_int = pd.DataFrame(columns = df_new.columns.tolist())
  for genre in genres:
    df_int =  pd.concat([df_int,df_new[df_new['Genre']==genre]]) 
  for index,row in df_int.iterrows():
    row['Director'] = row['Director'].split(",")[0]
  director_list1 = df_int['Director']
  directors = []
  director_list = director_list1.values
  director_list = list(director_list)
  for director in director_list:
    directors.append(director.split(',')[0])
  director_list = list(directors)
  director_set = set(director_list)
  df_int['Directors'] = directors
  count = 0
  director_dict={}
  director_list_i = []
  for director in director_set:
    try:
      director_dict[director] = np.array(wiki2vec.get_entity_vector(director))
    except KeyError:
      count = count + 1
      director_list_i.append(director)

  director_set_new = set(director_list_i)
  df_remove = pd.DataFrame(columns = df_int.columns.tolist())
  for director in director_set_new:
    df_remove =  pd.concat([df_remove,df_int[df_int['Director']==director]]) 
  df_int = df_int[~df_int.isin(df_remove)].dropna()
  df_int = df_int.loc[:,['Title','Genre','Plot','Directors']]
  df_final = pd.DataFrame(columns = df_int.columns.tolist())
  for genre in genres:
    df_final =  pd.concat([df_final,df_int[df_int['Genre']==genre][:700]]) 
  category_dict = {"drama":0,"comedy":1,"horror":2,"action":3,"thriller":4}
  df_final["Labels"] = df_final["Genre"].map(category_dict)
  df_final = df_final.sample(frac=1)
  df_final = df_final[['Title','Directors','Genre','Plot','Labels']]
  df_final.columns = ['Title','Director','Genre','Plot','Labels']
  df_final.to_csv('movie_genre5.csv', index = False)